In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import numpy as np
import pandas as pd
import sys
import datetime
import os
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import pickle
from collections import OrderedDict
import copy
from scipy.sparse import csr_matrix
from scipy import io
import seaborn as sns
import joblib
# from base import *
from joblib import Parallel, delayed
import random
import scipy

In [3]:
MNM_nb_folder = os.path.join('..', '..', '..', 'side_project', 'network_builder')
sys.path.append(MNM_nb_folder)
python_lib_folder = os.path.join('..', '..', 'pylib')
sys.path.append(python_lib_folder)

In [4]:
from MNMAPI import *
from MNM_mcnb import *
from mcDODE import *

In [6]:
data_folder = os.path.join('/home/lemma/Documents/MAC-POSTS/data/input_files_7link_multiclass')

In [7]:
nb = MNM_network_builder()
nb.load_from_folder(data_folder)

In [26]:
observed_link_list = [3, 4, 6]
ml_car = 2
ml_truck = 1
data_dict = dict()
num_interval = nb.config.config_dict['DTA']['max_interval']
true_car_x = np.random.rand(num_interval * len(observed_link_list))
true_truck_x = np.random.rand(num_interval * len(observed_link_list))
L_car_one = np.random.randint(2, size = (ml_car, len(observed_link_list)))
L_truck_one = np.random.randint(2, size = (ml_truck, len(observed_link_list)))
L_car = csr_matrix(scipy.linalg.block_diag(*[L_car_one for i in range(num_interval)]))
L_truck = csr_matrix(scipy.linalg.block_diag(*[L_truck_one for i in range(num_interval)]))
m_car = L_car.dot(true_car_x)
m_truck = L_truck.dot(true_truck_x)
data_dict['car_count_agg_L_list'] = [L_car]
data_dict['truck_count_agg_L_list'] = [L_truck]
data_dict['car_link_flow'] = [m_car]
data_dict['truck_link_flow'] = [m_truck]
# data_dict['car_link_tt'] = [m_spd_car]
# data_dict['truck_link_tt'] = [m_spd_truck]

In [27]:
config = dict()
config['use_car_link_flow'] = True
config['use_car_link_tt'] = False
config['use_truck_link_flow'] = True
config['use_truck_link_tt'] = False
config['car_count_agg'] = True
config['truck_count_agg'] = True
config['link_flow_weight'] = 1
config['link_tt_weight'] = 1
config['num_data'] = 1
config['observed_links'] = observed_link_list
config['paths_list'] = range(nb.config.config_dict['FIXED']['num_path'])

In [14]:
dode = MCDODE(nb, config)

In [28]:
dode.add_data(data_dict)